In [1]:
from os import times
import time
from pickletools import long1

from docutils.nodes import title
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from setuptools.command.install import install
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from trio import sleep
import requests
import base64
import pandas as pd
from urllib3 import request

import json
import os


In [56]:
def openDriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    DRIVER_PATH = "chromedriver.exe"
    return webdriver.Chrome(options=chrome_options, service=Service(DRIVER_PATH))


In [57]:
def getLink(uri):
    driver = openDriver()
    driver.get(uri)
    time.sleep(1)
    
    links = []
    parent = driver.find_elements(By.CLASS_NAME, "plp-fragment-wrapper")
    for index in parent:
        root = ((index.find_element(By.CLASS_NAME, "plp-mastercard ").
                 find_element(By.TAG_NAME, "a")).
                get_attribute("href"))
        links.append(root)
        
    driver.quit()
    return links


In [58]:
def saveDataToJSON(directory, filename, data):
    os.makedirs("json/link/" + directory, exist_ok=True)
    try:
        with open(f'json/link/{directory}/{filename}.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)
            
    except FileNotFoundError:
        existingData = []
    
    existingData.extend(data)
    with open(f'json/link/{directory}/{filename}.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
        

In [ ]:
# for i in range(12,14):
#     url = {
#         "furniture sets":
#                f"https://www.ikea.com/th/th/cat/furniture-sets-55036/?page={i}"
#     }
#     LinkList = getLink(url.get('furniture sets'))
#     saveDataToJSON("furniture sets","FurnitureSetsLink",LinkList)
#     print("page = ",i)
#     

In [59]:
def getLinkProduct(n, nameCat, urlCat):
    for i in range(1, n):
        uri = {
            nameCat:
                f"{urlCat}?page={i}"
        }
        linkList = getLink(uri.get(nameCat))
        saveDataToJSON(nameCat, f"{nameCat}Link", linkList)
        print("page = ", i)

In [60]:
def getDetailProduct(urlCat):
    driver = openDriver()
    driver.get(urlCat)
    
    allDetail = driver.find_element(By.XPATH, '//div[@class="js-price-package pip-price-package"]')
    brandName = allDetail.text.split("\n")[0]
    detail = allDetail.text.split("\n")[1].split(", ")
    productName = detail[0]
    try:
        color = detail[1]
    except:
        color = None
    try:
        size = detail[2]
    except:
        size = None
    price = allDetail.text.split("\n")[2][:-3]
    
    driver.quit()
    return brandName, productName,color, size, price

###กดปุ่ม เพื่อรับข้อมูล

In [61]:
def getDescFromProduct(uri):
    browser = openDriver()
    browser.get(uri)
    try:
        button = browser.find_element(By.CLASS_NAME, "pip-list-view-item__action")
        browser.execute_script("arguments[0].click();", button)
        
        paragraphs = browser.find_elements(By.CLASS_NAME, 'pip-product-details__paragraph')
        codeProduct = (browser.find_element(By.CLASS_NAME, 'pip-product-identifier__value').
                       get_attribute("innerText"))
        
        stringDesc = ""
        for paragraph in paragraphs:
            stringDesc += paragraph.get_attribute("innerText")+"\n\n"

        browser.quit()
        return stringDesc , codeProduct
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        browser.quit()

------------------------

In [62]:
# Download Image for Product
def saveImage(uri, dirName, fileName):
    response = requests.get(uri)
    
    if response.status_code == 200:
        os.makedirs("image/room/"+dirName, exist_ok=True)
        with open(f"image/room/{dirName}/{fileName}_image.jpg", "wb") as file:
            file.write(response.content)
    else:
        print("เกิดข้อผิดพลาดในการดาวน์โหลดภาพ:", response.status_code)


In [63]:
def kernelImage(dirName, fileName):
    with open(f"image/room/{dirName}/{fileName}_image.jpg", "rb") as png_file:
        # อ่านไฟล์ PNG
        png_data = png_file.read()
        # แปลงข้อมูลไฟล์ PNG เป็น Base64
        base64_encoded_data = base64.b64encode(png_data)
        # แปลง Base64 เป็น string เพื่อใช้งาน
        base64_string = base64_encoded_data.decode('utf-8')
        
    return base64_string
    
    

In [64]:
def getImage(uri, dirName, fileName):
    driver = openDriver()
    driver.get(uri)
    findElement = driver.find_element(By.XPATH, "//div[@class='pip-zoom-image']")
    img_element = findElement.find_element(By.TAG_NAME, "img")
    srcset = img_element.get_attribute("srcset")
    
    first_link = srcset.split(",")[0].strip().split(" ")[0]
    
    driver.quit()
    saveImage(first_link, dirName, fileName)
    return kernelImage(dirName, fileName)
    

In [11]:
# def readJson(pathname, filename):
#     data = pd.read_json("json/link/"+pathname+"/"+filename)
#     return data

In [65]:
def readJson(pathname, filename):
    data = pd.read_json("json/product/"+pathname+"/"+filename)
    return data

In [27]:
df = readJson("Beds","Beds_datas.json")

In [28]:
df = pd.json_normalize(df['Beds'])

In [29]:
df.drop(columns="imgBase64", inplace=True)

In [ ]:
df = df[df['brandName'] != 'ใหม่'].reset_index(drop=True)


In [ ]:

pattern = r'\d{1,4}x\d{1,4}\s?ซม\.?'

# ย้ายค่าที่ตรงกับ pattern จาก 'color' ไปยัง 'size'
df.loc[df['size'].isna() & df['color'].str.contains(pattern, na=False), 'size'] = df['color'].str.extract(f'({pattern})').iloc[:, 0]

# ลบค่าขนาดจากคอลัมน์ 'color'
df['color'] = df['color'].str.replace(pattern, 'None', regex=True).str.strip()
df['color']

In [14]:
# def saveProductDataToJson(dirName, fileName):
#     links = readJson(dirName, fileName)
#     datas = []
#     
#     try:
#         for i in links[0]:
#             brandName, productName,color, size, price = getDetailProduct(i)
#             stringDesc , codeProduct = getDescFromProduct(i)
#             imgBase64 = getImage(i, dirName, codeProduct)
#     
#             datas.append({
#                 "Code":codeProduct,
#                 "brandName":brandName,
#                 "productName":productName,
#                 "color":color,
#                 "size":size,
#                 "price":price,
#                 "imgBase64":imgBase64,
#                 "desc":stringDesc
#             })
#             
#         datas = {dirName:datas}
#         os.makedirs("json/product/" + dirName, exist_ok=True)
#         try:
#             with open(f'json/product/{dirName}/{dirName}_datas.json', 'w', encoding='utf-8') as FILE:
#                 json.dump(datas, FILE, ensure_ascii=False, indent=4)
#         except FileNotFoundError:
#             print(" Can not save data")
#             
#     except Exception as e:
#         print(e)

In [20]:
# saveProductDataToJson("dishwashing-accessories", "dishwashing-accessoriesLink.json")

1h 54m : 206 item

In [18]:
with open("json/link/baskets/basketsLink.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df

,0
0,https://www.ikea.com/th/th/p/branaes-basket-wh...
1,https://www.ikea.com/th/th/p/tjillevips-basket...
2,https://www.ikea.com/th/th/p/fladis-basket-sea...
3,https://www.ikea.com/th/th/p/bekna-basket-plas...
4,https://www.ikea.com/th/th/p/gnabbas-basket-40...
5,https://www.ikea.com/th/th/p/knipsa-basket-sea...
6,https://www.ikea.com/th/th/p/maevinn-basket-80...
7,https://www.ikea.com/th/th/p/bullig-box-bamboo...
8,https://www.ikea.com/th/th/p/lustigkurre-baske...
9,https://www.ikea.com/th/th/p/tolkning-basket-h...


In [19]:
df.columns = ['link']
df

,link
0,https://www.ikea.com/th/th/p/branaes-basket-wh...
1,https://www.ikea.com/th/th/p/tjillevips-basket...
2,https://www.ikea.com/th/th/p/fladis-basket-sea...
3,https://www.ikea.com/th/th/p/bekna-basket-plas...
4,https://www.ikea.com/th/th/p/gnabbas-basket-40...
5,https://www.ikea.com/th/th/p/knipsa-basket-sea...
6,https://www.ikea.com/th/th/p/maevinn-basket-80...
7,https://www.ikea.com/th/th/p/bullig-box-bamboo...
8,https://www.ikea.com/th/th/p/lustigkurre-baske...
9,https://www.ikea.com/th/th/p/tolkning-basket-h...


In [20]:
with open("json/link/Bed Textile/dropDupBedTextileLink.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

df2 = pd.DataFrame(data)
df2

,link
0,https://www.ikea.com/th/th/p/dvala-fitted-shee...
1,https://www.ikea.com/th/th/p/stjaernstarr-duve...
2,https://www.ikea.com/th/th/p/bruksvara-4-piece...
3,https://www.ikea.com/th/th/p/grusstarr-duvet-c...
4,https://www.ikea.com/th/th/p/slanhoestmal-duve...
...,...
812,https://www.ikea.com/th/th/p/sandeternell-cush...
813,https://www.ikea.com/th/th/p/soetroenn-cushion...
814,https://www.ikea.com/th/th/p/bruntag-cushion-c...
815,https://www.ikea.com/th/th/p/jaettevallmo-fitt...


True


In [49]:
directory_path = 'json/link'

# สร้าง DataFrame ว่างเพื่อเก็บข้อมูลทั้งหมด
all_data = pd.DataFrame()

# ใช้ os.walk เพื่อเดินทางในไดเรกทอรี่ทั้งหมด
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.json'):
            # สร้าง path ของไฟล์ JSON
            file_path = os.path.join(root, file)

            # ดึงชื่อโฟลเดอร์
            folder_name = os.path.basename(root)

            # อ่านข้อมูลจากไฟล์ JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # แปลงข้อมูลเป็น DataFrame และเก็บข้อมูล folder_name
                df = pd.DataFrame(data, columns=['link'])
                df['folder_name'] = folder_name  # เพิ่มคอลัมน์ใหม่ที่เก็บชื่อโฟลเดอร์

                # นำ DataFrame ไปต่อรวมกับ all_data
                all_data = pd.concat([all_data, df], ignore_index=True)

In [ ]:
all_data

In [55]:
a = "https://www.ikea.com/th/th/p/branaes-basket-white-20393412/"

if a in all_data['link'].values:
    print(5)
else:
    print(6)
    

5


In [66]:
def getB64ProductImg(uri):
    driver = openDriver()
    driver.get(uri)
    findElement = driver.find_element(By.XPATH, "//div[@class='pip-zoom-image']")
    img_element = findElement.find_element(By.TAG_NAME, "img")
    srcset = img_element.get_attribute("srcset")
    first_link = srcset.split(",")[0].strip().split(" ")[0]
    driver.quit()
    
    imgB64 = requests.get(first_link).content
    imgB64 = base64.b64encode(imgB64)
    imgB64 = imgB64.decode("utf-8")
    return imgB64

In [73]:
def getProductDataInSubImg(element, all_data = all_data):
    element = element.find_elements(By.CLASS_NAME, "pub__shoppable-image__dot")
    linkProduct = [i.get_attribute("href") for i in element]
    code = []
    dataSet = []
    
    for i in linkProduct:
        brandName, productName,color, size, price = getDetailProduct(i)
        stringDesc , codeProduct = getDescFromProduct(i)
        imgBase64 = getB64ProductImg(i)
        cate = ""
        
        if i in all_data['link'].values:
            dataSet.append({
                "cate": all_data['folder_name'].values[i],
                "Code":codeProduct,
                "brandName":brandName,
                "productName":productName,
                "color":color,
                "size":size,
                "price":price,
                "imgBase64":imgBase64,
                "desc":stringDesc
            })

            os.makedirs("json/product/product-in-subImg", exist_ok=True)
            try:
                with open('json/product/product-in-subImg/product-in-subImg-data.json', 'r', encoding='utf-8') as FILE:
                    existingData = json.load(FILE)
    
            except FileNotFoundError:
                existingData = []
    
            existingData.extend(dataSet)
            with open('json/product/product-in-subImg/product-in-subImg-data.json', 'w', encoding='utf-8') as FILE:
                json.dump(existingData, FILE, ensure_ascii=False, indent=4)
            
        else:
            dataSet.append({
                "cate": cate,
                "Code":codeProduct,
                "brandName":brandName,
                "productName":productName,
                "color":color,
                "size":size,
                "price":price,
                "imgBase64":imgBase64,
                "desc":stringDesc
            })
            
            os.makedirs("json/product/no-category-data", exist_ok=True)
            try:
                with open('json/product/no-category-data/no-categot-product-data.json', 'r', encoding='utf-8') as FILE:
                    existingData = json.load(FILE)
            
            except FileNotFoundError:
                existingData = []
            
            existingData.extend(dataSet)
            with open('json/product/no-category-data/no-categot-product-data.json', 'w', encoding='utf-8') as FILE:
                json.dump(existingData, FILE, ensure_ascii=False, indent=4)
            
        code.append(codeProduct)
    return code

In [74]:
def getSubImgInRoom(driver):
    subImgInRoom = (driver.
                   find_elements(By.XPATH, 
                   "//div[@class='pub__shoppable-image pub__shoppable-image--visible-dots']")
                   )
    dataSet = []
    idSet = []
    j = 0
    
    for i in subImgInRoom:
        id = j
        j = j + 1
        
        subImg = i.find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]
        subImg = requests.get(subImg).content
        subImg = base64.b64encode(subImg)
        subImg = subImg.decode('utf-8')
        
        productCode = [].append(getProductDataInSubImg(i))
        
        dataSet.append({
            "id":id,
            "subImg":subImg,
            "productCode":productCode
        })
        idSet.append(id)

    os.makedirs("json/room", exist_ok=True)
    try:
        with open('json/room/sub-img-rooms-data.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)

    except FileNotFoundError:
        existingData = []

    existingData.extend(dataSet)
    with open('json/room/sub-img-rooms-data.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
    
        
    return idSet
    

In [75]:
def saveDataInRoom(uri):
    browser = openDriver()
    browser.get(uri)
    body = browser.find_element(By.TAG_NAME, "body")
    
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        
    titleDescRoom = browser.find_element(By.XPATH, "//h1[@class='c19ofv4i pub__h1 s1gshh7t']").text
    desc = browser.find_element(By.XPATH, "//div[@class='i1ycpxq9 m1e9rlhw pub__designSystemText t91kxqv w1fdzi2f']").text
    mainImage = browser.find_element(By.XPATH, "//span[@class='pub__aspect-ratio-box pub__aspect-ratio-box--wide']").find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]
    
    mainImage = requests.get(mainImage).content
    mainImage = base64.b64encode(mainImage)
    mainImage = mainImage.decode("utf-8") #base64 in form string
    
    idSubImg = [].append(getSubImgInRoom(browser))
    browser.quit()
    
    return titleDescRoom, desc, mainImage, idSubImg

    

In [77]:
def saveRoomData(uriRoom, roomName):
    titleDescRoom, desc, mainImage, idSubImg = saveDataInRoom(uriRoom)
    roomData = [
        {
            "roomName": roomName,
            "title":titleDescRoom,
            "desc":desc,
            "mainImage":mainImage,
            "idSubImg":idSubImg        
        }
    ]
    
    os.makedirs("json/room", exist_ok=True)
    try:
        with open('json/room/rooms-data.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)

    except FileNotFoundError:
        existingData = []

    existingData.extend(roomData)
    with open('json/room/rooms-data.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
        

In [ ]:
saveRoomData("https://www.ikea.com/th/th/rooms/living-room/gallery/a-retro-living-room-with-lots-of-storage-pubfe564a00", "living-room")

In [140]:
dirName = {
    "bedroom":{
        "titleDescRoom" : nameDesc,
        "desc" : desc,
        "mainImageB64" : 1,
        "idImgSet" :{0,1,2}
    },
    "bathroom":{
        "titleDescRoom" : nameDesc,
        "desc" : desc,
        "mainImageB64" : 2,
        "idImgSet" :{0,1,2}
    }
}

ImgItemInRoom = [
    {   
        'id':0,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    },
    {
        'id':1,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    },
    {
        'id':2,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    }
]

dirItemInRoom = {
    "cateName":[
        {
            "codeId" : 1,
            "brandName": "app",
            "productName": "lamp",
            "color": "หรี่ไฟได้ ขาว",
            "size": "80 ซม.",
            "price": "790บาท/ช",
            "imageB64" : "b64",
            "desc":"hello world"
        },
        {
            "codeId" : 2,
            "brandName": "app",
            "productName": "lamp",
            "color": "หรี่ไฟได้ ขาว",
            "size": "80 ซม.",
            "price": "790บาท/ช",
            "imageB64" : "b64",
            "desc":"hello world"
        }
    ]
}

---------

In [83]:
driver = openDriver()

In [84]:
driver.get("https://www.ikea.com/th/th/rooms/bedroom/gallery/a-dreamy-tropical-bohemian-style-teenagers-bedroom-pub0ecd6380")
body = driver.find_element(By.TAG_NAME, "body")
for i in range(10):  # ปรับค่าตามจำนวนครั้งที่ต้องการเลื่อน
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)  # รอให้ข้อมูลโหลด

In [57]:
productRoom = driver.find_elements(By.XPATH, "//div[@class='pub__shoppable-image pub__shoppable-image--visible-dots']")

In [122]:
nameDesc = driver.find_element(By.XPATH, "//h1[@class='c19ofv4i pub__h1 s1gshh7t']").text
desc = (driver.find_element(By.XPATH, "//div[@class='i1ycpxq9 m1e9rlhw pub__designSystemText t91kxqv w1fdzi2f']")
        .text)
mainImage = driver.find_element(By.XPATH, "//span[@class='pub__aspect-ratio-box pub__aspect-ratio-box--wide']").find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]


In [58]:
linkBut = {}
for i in productRoom:
    x = i.find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]
    # link.append(x)
    y = i.find_elements(By.CLASS_NAME, "pub__shoppable-image__dot")
    linkBut[x] = [j.get_attribute("href") for j in y]


In [ ]:
# linkBut

In [ ]:
d = {
    "bed":{
    {
    "ID":"1233",
    "name":"hello",
    "price":1,
    "size":1,
    "desc":"Hi",
    "color":"blue",
    "stock":1,
    "img":"base64"
    },
    {
    "name":"hello",
    "price":1,
    "size":1,
    "desc":"Hi",
    "color":"blue",
    "stock":1,
    "img":"base64"
    }
    }
}